In [1]:
import face_recognition
import os

def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]

    return cropped_face

In [5]:
dir_path = os.getenv('HOME')+'/aiffel/face_embedding/idol'
file_list = os.listdir(dir_path)

print ("file_list: {}".format(file_list))

file_list: ['리사.jpg', '태연.jpeg', '유리.jpg', '유아.jpeg', '지수.jpeg', '써니.jpg', '이효리.jpg', '최유정.jpeg', '하영.jpeg', '조이.jpeg', '우기.jpg', '윤아.jpeg', '러블리즈예인.jpg', '권은비.jpeg', '안유진.jpeg', '잇지유나.jpg', '도아.jpeg', '써니1.jpeg', '청하.jpeg', '아이린.jpeg', '화사.jpeg', '잇지예지.jpeg', '선미.jpeg', '전소미.jpg', '나인뮤지스경리.jpg', '미미.jpg', '사나.jpeg', '여자아이들수진.jpeg', '효연.jpeg', '효정.jpeg', '케이.jpg', '슬기.jpeg', '미나.jpeg', '트와이스모모.jpeg', '혜리.jpeg', '아이즈원민주.jpg', '지효.jpeg', '채연.jpg', '류진.jpg', '아이유.jpeg', '제시카.jpeg', '사쿠라.jpeg', '휘인.jpeg', '웬디.jpeg', '수영.jpeg', '아린.jpeg', '설현.jpeg', '티파니.jpg', '문별.jpeg', '크리스탈.jpeg', '승희.jpeg', '제니.jpeg', '비니.jpg', '레인보우재경.jpg', '민니.jpg', '예리.jpeg', '다현.jpeg', '수지.jpeg', '로제.jpg', '나연.jpeg', '리아.jpg', '은지.jpeg', '정연.jpeg', '신비.jpg', '솔라.jpeg']


In [8]:
def get_face_embedding(face):
    return face_recognition.face_encodings(face)


def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}

    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_gropped_face(img_path)
        embedding = get_face_embedding(face)
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
                    # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]

    return embedding_dict

In [9]:
embedding_dict = get_face_embedding_dict(dir_path)

In [10]:
embedding_dict.keys()

dict_keys(['리사', '태연', '유리', '유아', '지수', '써니', '이효리', '최유정', '하영', '조이', '우기', '윤아', '러블리즈예인', '권은비', '안유진', '잇지유나', '써니1', '청하', '아이린', '화사', '잇지예지', '선미', '전소미', '나인뮤지스경리', '미미', '사나', '여자아이들수진', '효연', '효정', '케이', '슬기', '미나', '트와이스모모', '혜리', '아이즈원민주', '지효', '채연', '류진', '아이유', '사쿠라', '휘인', '웬디', '수영', '아린', '설현', '티파니', '문별', '크리스탈', '승희', '제니', '비니', '레인보우재경', '민니', '예리', '다현', '수지', '로제', '나연', '리아', '은지', '정연', '솔라'])

In [12]:
my1 = embedding_dict['써니']
my2 = embedding_dict['써니1']

In [14]:
import numpy as np
print(np.linalg.norm(my2-my1, ord=1))  #L1 distance
print(np.linalg.norm(my2-my1, ord=2))  #L2 distance

3.6546754455193877
0.4201745583932428


In [18]:
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

sort_key_func = get_sort_key_func('써니')   

In [19]:
sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))

[('써니',
  array([-7.81238526e-02,  8.96422341e-02, -3.45246792e-02, -6.92325234e-02,
         -1.28715128e-01, -3.86847109e-02, -6.67029023e-02, -9.68407169e-02,
          1.29210785e-01, -5.94233721e-02,  2.29590118e-01, -1.14343531e-01,
         -2.28509575e-01, -5.71648367e-02, -4.04544324e-02,  2.26661474e-01,
         -1.77418292e-01, -1.79793283e-01, -5.96532412e-02, -9.85016301e-03,
          1.05027921e-01, -7.67176971e-03, -2.60034576e-03,  1.40295506e-01,
         -1.41394541e-01, -3.33363652e-01, -8.68085623e-02, -1.34496123e-01,
         -6.80881739e-02, -1.13045968e-01, -1.61544457e-02,  4.16748412e-02,
         -2.23655939e-01, -4.96967398e-02,  5.72695732e-02,  6.26403391e-02,
         -5.80617599e-03, -9.35165882e-02,  1.86602548e-01, -2.84371898e-04,
         -2.89851636e-01,  5.00009321e-02,  1.36453241e-01,  2.28462890e-01,
          1.32733658e-01,  2.53468156e-02, -3.28987837e-03, -1.52804956e-01,
          1.18009582e-01, -1.37482300e-01,  8.02820399e-02,  1.63425

In [22]:
def get_nearest_face(name, top=5):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))

    for i in range(top+1):
        if i == 0 :# 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))

In [23]:
get_nearest_face('써니')

써니
순위 1 : 이름(혜리), 거리(0.3738187184209193)
순위 2 : 이름(써니1), 거리(0.4201745583932428)
순위 3 : 이름(하영), 거리(0.4293634578844898)
순위 4 : 이름(지수), 거리(0.4355614049297601)
순위 5 : 이름(설현), 거리(0.43978184561043643)
